# Chapter 7

In [2]:
import sys
import numpy as np
import pandas as pd
import csv
import json
from lxml import objectify
import pyarrow
import openpyxl
import xlrd
from bs4 import BeautifulSoup
import sqlalchemy as sqla
import sqlite3
import os
from pathlib import Path

### Handling Missing Data

In [3]:
float_data = pd.Series([1.2, -3.5, np.nan, 0])

In [4]:
float_data

0    1.2
1   -3.5
2    NaN
3    0.0
dtype: float64

In [5]:
float_data.isna()

0    False
1    False
2     True
3    False
dtype: bool

In [6]:
string_data = pd.Series(["aardvark", np.nan, None, "avocado"])

In [7]:
string_data

0    aardvark
1         NaN
2        None
3     avocado
dtype: object

In [8]:
string_data.isna()

0    False
1     True
2     True
3    False
dtype: bool

In [9]:
float_data = pd.Series([1,2,None], dtype='float64')

In [10]:
float_data

0    1.0
1    2.0
2    NaN
dtype: float64

In [11]:
float_data.isna()

0    False
1    False
2     True
dtype: bool

## NA Handling Objects Methods<br>
**dropna**<br>
*Filter axis labels based on whether the values for each label have missing datd , with varying threshholds for how much missing data to tolerate.* <br>
**fillna**<br>
*Fill in missing data with some value or using an interpolation method such as "ffill" or "bfill".* <br>
**isna**<br>
*Return Boolean values indicating which values are missing/NA.* <br>
**notna**<br>
*Negation of isna, returns True for non_NA values and False for NA values.

### Filtering out missing data

In [12]:
data = pd.Series([1, np.nan, 3.5, np.nan, 7])

In [13]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [14]:
data[data.notna()]

0    1.0
2    3.5
4    7.0
dtype: float64

In [17]:
data = pd.DataFrame([[1., 6.5, 3.], [1., np.nan, np.nan], [np.nan, np.nan, np.nan],[np.nan, 6.5, 3.]])

In [18]:
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [20]:
data.dropna(axis="columns", how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [21]:
df = pd.DataFrame(np.random.standard_normal((7,3)))

In [22]:
df.iloc[:4, 1] = np.nan

In [23]:
df.iloc[:2, 2] = np.nan

In [24]:
df

,0,1,2
0,-0.226830,NaN,NaN
1,-0.008607,NaN,NaN
2,-0.407546,NaN,-0.238661
3,1.569548,NaN,0.103098
4,-0.111287,-0.353035,-0.161954
5,0.553252,0.840681,-0.217661
6,-1.167717,0.928221,2.165731


In [25]:
df.dropna()

,0,1,2
4,-0.111287,-0.353035,-0.161954
5,0.553252,0.840681,-0.217661
6,-1.167717,0.928221,2.165731


In [26]:
df.dropna(thresh=2)

,0,1,2
2,-0.407546,NaN,-0.238661
3,1.569548,NaN,0.103098
4,-0.111287,-0.353035,-0.161954
5,0.553252,0.840681,-0.217661
6,-1.167717,0.928221,2.165731
